In [17]:
file_path = '/Volumes/EXTERNALHDD/traces_disk_2/final_data_files/data_1.parquet'

In [18]:
import pandas as pd

edges_df = pd.read_parquet(file_path)

In [19]:
edges_df

,u,i,ts,idx,rt,0,1,2,3,4
3797497,17684,18738,60000,3797498,1.0,0.0,0.0,1.0,0.0,0.0
3797498,35482,19355,60000,3797499,0.0,0.0,0.0,0.0,0.0,1.0
3797499,18679,59226,60000,3797500,1.0,0.0,0.0,1.0,0.0,0.0
3797500,33545,14046,60000,3797501,0.0,0.0,0.0,1.0,0.0,0.0
3797501,969,17869,60000,3797502,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7393462,10077,35136,119999,7393463,0.0,0.0,0.0,1.0,0.0,0.0
7393463,44631,44354,119999,7393464,1.0,0.0,0.0,0.0,0.0,1.0
7393464,37176,37400,119999,7393465,1.0,0.0,0.0,1.0,0.0,0.0
7393465,37106,22350,119999,7393466,0.0,1.0,0.0,0.0,0.0,0.0


In [20]:
u, i, ts = edges_df['u'].to_numpy(), edges_df['i'].to_numpy(), edges_df['ts'].to_numpy()

In [21]:
from temporal_random_walk import TemporalRandomWalk

temporal_random_walk = TemporalRandomWalk(is_directed=True, use_gpu=False)

In [22]:
from raphtory import Graph
g = Graph()

In [23]:
g.load_edges_from_pandas(
    df=edges_df,
    time="ts",
    src="u",
    dst="i"
)

In [24]:
temporal_random_walk.add_multiple_edges(u, i, ts)

In [26]:
import pandas as pd

df = pd.read_parquet('/Volumes/EXTERNALHDD/traces_disk_2/final_data_files/data_100.parquet')

In [28]:
df

,u,i,ts,idx,rt,0,1,2,3,4
335204989,14938,14938,6000000,335204990,3.0,0.0,0.0,0.0,0.0,1.0
335204990,16284,17932,6000000,335204991,0.0,0.0,0.0,1.0,0.0,0.0
335204991,43759,28205,6000000,335204992,1.0,0.0,0.0,1.0,0.0,0.0
335204992,66999,66999,6000000,335204993,NaN,0.0,0.0,1.0,0.0,0.0
335204993,5870,2402,6000000,335204994,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
338246044,28012,37927,6059999,338246045,1.0,1.0,0.0,0.0,0.0,0.0
338246045,51804,20354,6059999,338246046,0.0,0.0,0.0,0.0,1.0,0.0
338246046,32740,32740,6059999,338246047,0.0,0.0,0.0,0.0,0.0,1.0
338246047,67001,31913,6059999,338246048,6.0,0.0,1.0,0.0,0.0,0.0
